In [1]:
import numpy as np
import tensorflow as tf
from sklearn.manifold import TSNE
from dataset import Dataset
from model import Model
import matplotlib.pyplot as plt
from sklearn.metrics.pairwise import cosine_similarity
from dataset import Dataset
from scipy.io import savemat

In [2]:
def sample_z(m, n):
    return np.random.normal(loc=0., scale=1., size=[m, n])
    #return np.random.uniform(-5., 5., size=[m, n])

In [3]:
dt = Dataset('/home/kt/NIST_npy/', one_hot=True)
x_dim = dt.train_data.shape[1]
y_dim = dt.train_label.shape[1]
z_dim = 100
save_path = "save_good/model.ckpt"

In [4]:
model = Model(x_dim, z_dim, y_dim)
saver = tf.train.Saver()
sess = tf.Session()
saver.restore(sess, save_path)
print("Model restored")

INFO:tensorflow:Restoring parameters from save_good/model.ckpt
Model restored


In [27]:
for i in range(1, 11, 1):
    x = np.where(dt.bincout < i*2)[0]
    print x.shape[0], 'classes less than', i*2 
n = 2
count = n - dt.bincout
idx = np.where(count > 0)[0]
total = count[idx]
gen_sum = np.sum(total)
y = []
for i in range(len(idx)):
    for j in range(total[i]):
        y.append(idx[i])
y = np.asarray(y).astype(np.int)
print len(idx), len(total), gen_sum, len(y)

one_hot = np.zeros([gen_sum, y_dim], dtype=np.int)
one_hot[np.arange(len(y)), y] = 1
z = sample_z(len(y), z_dim)

1033 classes less than 2
1990 classes less than 4
2445 classes less than 6
2760 classes less than 8
3277 classes less than 10
3804 classes less than 12
4174 classes less than 14
4417 classes less than 16
4594 classes less than 18
4711 classes less than 20
1033 1033 1033 1033


In [28]:
for i in np.argmax(one_hot, axis=1) == y:
    if i != True:
        print 'yes'

In [29]:
gen_x = model.generate(sess, z, one_hot)

In [30]:
np.save('Gen_Data/cgen_x_' + str(n) + '.npy', gen_x)
np.save('Gen_Data/cgen_y_' + str(n) + '.npy', y)
np.save('Gen_Data/cgen_y_onthot_' + str(n) + '.npy', one_hot)

In [31]:
dt = Dataset('/home/kt/NIST_npy/', one_hot=False)

In [32]:
print dt.train_data.shape
print dt.train_label.shape

(36572, 600)
(36572,)


In [33]:
print gen_x.shape
print y.shape

(1033, 600)
(1033,)


In [34]:
dt.add_training_data(gen_x, y)
print dt.train_data.shape
print dt.train_label.shape

Adding training data
(37605, 600)
(37605,)


In [35]:
dt.sort()
mat_x = dt.train_data
mat_y = dt.train_label + 1

sorting


In [36]:
mat_y[0:10]

array([1, 1, 2, 2, 3, 3, 4, 4, 5, 5])

In [37]:
train_x = {'x': mat_x}
train_y = {'y': mat_y}

In [38]:
savemat('save_mat/pgen_x_' + str(n) + '.mat', train_x)
savemat('save_mat/pgen_y_' + str(n) + '.mat', train_y)